In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('minadz').getOrCreate()

In [2]:
data = spark.read.csv('Malware_Classification.csv', inferSchema=True, header=True)

In [3]:
data.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- md5: string (nullable = true)
 |-- Machine: string (nullable = true)
 |-- SizeOfOptionalHeader: integer (nullable = true)
 |-- Characteristics: integer (nullable = true)
 |-- MajorLinkerVersion: integer (nullable = true)
 |-- MinorLinkerVersion: integer (nullable = true)
 |-- SizeOfCode: long (nullable = true)
 |-- SizeOfInitializedData: long (nullable = true)
 |-- SizeOfUninitializedData: long (nullable = true)
 |-- AddressOfEntryPoint: long (nullable = true)
 |-- BaseOfCode: long (nullable = true)
 |-- BaseOfData: integer (nullable = true)
 |-- ImageBase: double (nullable = true)
 |-- SectionAlignment: integer (nullable = true)
 |-- FileAlignment: integer (nullable = true)
 |-- MajorOperatingSystemVersion: integer (nullable = true)
 |-- MinorOperatingSystemVersion: integer (nullable = true)
 |-- MajorImageVersion: integer (nullable = true)
 |-- MinorImageVersion: integer (nullable = true)
 |-- MajorSubsystemVersion: integer (nullable = tru

In [4]:
data.summary

<bound method DataFrame.summary of DataFrame[ID: int, md5: string, Machine: string, SizeOfOptionalHeader: int, Characteristics: int, MajorLinkerVersion: int, MinorLinkerVersion: int, SizeOfCode: bigint, SizeOfInitializedData: bigint, SizeOfUninitializedData: bigint, AddressOfEntryPoint: bigint, BaseOfCode: bigint, BaseOfData: int, ImageBase: double, SectionAlignment: int, FileAlignment: int, MajorOperatingSystemVersion: int, MinorOperatingSystemVersion: int, MajorImageVersion: int, MinorImageVersion: int, MajorSubsystemVersion: int, MinorSubsystemVersion: int, SizeOfImage: int, SizeOfHeaders: int, CheckSum: bigint, Subsystem: int, DllCharacteristics: int, SizeOfStackReserve: double, SizeOfStackCommit: double, SizeOfHeapReserve: bigint, SizeOfHeapCommit: int, LoaderFlags: bigint, NumberOfRvaAndSizes: bigint, SectionsNb: int, SectionsMeanEntropy: double, SectionsMinEntropy: double, SectionsMaxEntropy: double, SectionsMeanRawsize: double, SectionsMinRawsize: int, SectionMaxRawsize: bigint

In [5]:
data = data.drop("ID", "md5", "Machine", "_c57", )

In [6]:
data.summary

<bound method DataFrame.summary of DataFrame[SizeOfOptionalHeader: int, Characteristics: int, MajorLinkerVersion: int, MinorLinkerVersion: int, SizeOfCode: bigint, SizeOfInitializedData: bigint, SizeOfUninitializedData: bigint, AddressOfEntryPoint: bigint, BaseOfCode: bigint, BaseOfData: int, ImageBase: double, SectionAlignment: int, FileAlignment: int, MajorOperatingSystemVersion: int, MinorOperatingSystemVersion: int, MajorImageVersion: int, MinorImageVersion: int, MajorSubsystemVersion: int, MinorSubsystemVersion: int, SizeOfImage: int, SizeOfHeaders: int, CheckSum: bigint, Subsystem: int, DllCharacteristics: int, SizeOfStackReserve: double, SizeOfStackCommit: double, SizeOfHeapReserve: bigint, SizeOfHeapCommit: int, LoaderFlags: bigint, NumberOfRvaAndSizes: bigint, SectionsNb: int, SectionsMeanEntropy: double, SectionsMinEntropy: double, SectionsMaxEntropy: double, SectionsMeanRawsize: double, SectionsMinRawsize: int, SectionMaxRawsize: bigint, SectionsMeanVirtualsize: double, Sect

### Find missing values

In [7]:
from pyspark.sql.functions import isnull, when, count, col
nacounts = data.select([count(when(isnull(c), c)).alias(c) for c in data.columns]).toPandas()
nacounts

,SizeOfOptionalHeader,Characteristics,MajorLinkerVersion,MinorLinkerVersion,SizeOfCode,SizeOfInitializedData,SizeOfUninitializedData,AddressOfEntryPoint,BaseOfCode,BaseOfData,...,ResourcesNb,ResourcesMeanEntropy,ResourcesMinEntropy,ResourcesMaxEntropy,ResourcesMeanSize,ResourcesMinSize,ResourcesMaxSize,LoadConfigurationSize,VersionInformationSize,legitimate
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
from pyspark.ml.feature import Imputer
imputer = Imputer(strategy='mean', inputCols=['MajorLinkerVersion'], outputCols=['MajorLinkerVersionImputed'])
imputer_model = imputer.fit(data)
data = imputer_model.transform(data)

In [9]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["SizeOfOptionalHeader",  "Characteristics", "MajorLinkerVersionImputed",  "MinorLinkerVersion", "SizeOfCode", "SizeOfInitializedData", "SizeOfUninitializedData", "AddressOfEntryPoint", "BaseOfCode", "BaseOfData", "ImageBase", "SectionAlignment", "FileAlignment", "MajorOperatingSystemVersion", "MinorOperatingSystemVersion", "MajorImageVersion", "MinorImageVersion", "MajorSubsystemVersion", "MinorSubsystemVersion", "SizeOfImage", "SizeOfHeaders", "CheckSum", "Subsystem", "DllCharacteristics", "SizeOfStackReserve", "SizeOfStackCommit", "SizeOfHeapReserve", "SizeOfHeapCommit", "LoaderFlags", "NumberOfRvaAndSizes", "SectionsNb", "SectionsMeanEntropy", "SectionsMinEntropy", "SectionsMaxEntropy", "SectionsMeanRawsize", "SectionsMinRawsize", "SectionMaxRawsize", "SectionsMeanVirtualsize", "SectionsMinVirtualsize", "SectionMaxVirtualsize", "ImportsNbDLL", "ImportsNb", "ImportsNbOrdinal", "ExportNb", "ResourcesNb", "ResourcesMeanEntropy", "ResourcesMinEntropy", "ResourcesMaxEntropy", "ResourcesMeanSize", "ResourcesMinSize", "ResourcesMaxSize", "LoadConfigurationSize", "VersionInformationSize"], outputCol='features')
data = assembler.transform(data)

In [10]:
#data = data.select(['features', "SizeOfOptionalHeader",  "Characteristics", "MajorLinkerVersionImputed",  "MinorLinkerVersion", "SizeOfCode", "SizeOfInitializedData", "SizeOfUninitializedData", "AddressOfEntryPoint", "BaseOfCode", "BaseOfData", "ImageBase", "SectionAlignment", "FileAlignment", "MajorOperatingSystemVersion", "MinorOperatingSystemVersion", "MajorImageVersion", "MinorImageVersion", "MajorSubsystemVersion", "MinorSubsystemVersion", "SizeOfImage", "SizeOfHeaders", "CheckSum", "Subsystem", "DllCharacteristics", "SizeOfStackReserve", "SizeOfStackCommit", "SizeOfHeapReserve", "SizeOfHeapCommit", "LoaderFlags", "NumberOfRvaAndSizes", "SectionsNb", "SectionsMeanEntropy", "SectionsMinEntropy", "SectionsMaxEntropy", "SectionsMeanRawsize", "SectionsMinRawsize", "SectionMaxRawsize", "SectionsMeanVirtualsize", "SectionsMinVirtualsize", "SectionMaxVirtualsize", "ImportsNbDLL", "ImportsNb", "ImportsNbOrdinal", "ExportNb", "ResourcesNb", "ResourcesMeanEntropy", "ResourcesMinEntropy", "ResourcesMaxEntropy", "ResourcesMeanSize", "ResourcesMinSize", "ResourcesMaxSize", "LoadConfigurationSize", "VersionInformationSize",'legitimate'])

In [11]:
data = data.select(['features', 'legitimate'])

In [12]:
train_df, test_df = data.randomSplit([0.7, 0.3])

In [13]:
from pyspark.ml.classification import RandomForestClassifier
algo = RandomForestClassifier(featuresCol='features', labelCol='legitimate')
model = algo.fit(train_df)

In [14]:
predictions = model.transform(test_df)

In [15]:
predictions.select(['legitimate','prediction', 'probability']).show()

+----------+----------+--------------------+
|legitimate|prediction|         probability|
+----------+----------+--------------------+
|         1|       0.0|[0.95158184768211...|
|         1|       0.0|[0.68079237973829...|
|         1|       0.0|[0.91056105431035...|
|         1|       0.0|[0.67233476361631...|
|         1|       0.0|[0.96804068299810...|
|         1|       0.0|[0.93465787042471...|
|         1|       0.0|[0.97135549388957...|
|         1|       0.0|[0.93873568903949...|
|         1|       0.0|[0.95473616762003...|
|         1|       0.0|[0.76844392905985...|
|         1|       0.0|[0.88544421989693...|
|         1|       0.0|[0.64977234542762...|
|         1|       0.0|[0.67971891791698...|
|         1|       0.0|[0.67823651043939...|
|         1|       0.0|[0.72961451362673...|
|         1|       0.0|[0.72961451362673...|
|         1|       0.0|[0.67971891791698...|
|         1|       0.0|[0.72813210614914...|
|         1|       0.0|[0.72813210614914...|
|         

In [16]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# Create both evaluators
evaluatorMulti = MulticlassClassificationEvaluator(labelCol="legitimate", predictionCol="prediction")
evaluator = BinaryClassificationEvaluator(labelCol="legitimate", rawPredictionCol="prediction", metricName='areaUnderROC')

# Make predicitons
predictionAndTarget = model.transform(test_df).select("legitimate", "prediction")

# Get metrics
acc = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "accuracy"})
f1 = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "f1"})
weightedPrecision = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedPrecision"})
weightedRecall = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedRecall"})
auc = evaluator.evaluate(predictionAndTarget)

In [17]:
print("accuracy score %.2f" % acc)
print("f1 score %.2f" % f1)
print("weighted precision %.2f" % weightedPrecision)
print("weighted recall score %.2f" % weightedRecall)
print("area under roc score %.2f" % auc)

accuracy score 0.96
f1 score 0.96
weighted precision 0.96
weighted recall score 0.96
area under roc score 0.96


In [18]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# Make predicitons
predictionAndTarget = model.transform(test_df).select("legitimate", "prediction")

predictionAndTargetNumpy = np.array((predictionAndTarget.collect()))

acc = accuracy_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
f1 = f1_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
precision = precision_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
recall = recall_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
auc = roc_auc_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])

In [19]:
print("accuracy score %.2f" % acc)
print("f1 score %.2f" % f1)
print("weighted precision %.2f" % precision)
print("weighted recall score %.2f" % recall)
print("area under roc score %.2f" % auc)

accuracy score 0.96
f1 score 0.94
weighted precision 0.94
weighted recall score 0.95
area under roc score 0.96


In [20]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, featuresCol='features', labelCol='legitimate')

# Fit the model
lrModel = lr.fit(train_df)

In [21]:
# Create both evaluators
evaluatorMulti = MulticlassClassificationEvaluator(labelCol="legitimate", predictionCol="prediction")
evaluator = BinaryClassificationEvaluator(labelCol="legitimate", rawPredictionCol="prediction", metricName='areaUnderROC')

# Make predicitons
predictionAndTarget = lrModel.transform(test_df).select("legitimate", "prediction")

# Get metrics
acc = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "accuracy"})
f1 = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "f1"})
weightedPrecision = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedPrecision"})
weightedRecall = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedRecall"})
auc = evaluator.evaluate(predictionAndTarget)

print("accuracy score %.2f" % acc)
print("f1 score %.2f" % f1)
print("weighted precision %.2f" % weightedPrecision)
print("weighted recall score %.2f" % weightedRecall)
print("area under roc score %.2f" % auc)

accuracy score 0.65
f1 score 0.51
weighted precision 0.42
weighted recall score 0.65
area under roc score 0.50


In [22]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(labelCol="legitimate", featuresCol="features")
dtModel = dt.fit(train_df)

In [23]:
evaluatorMulti = MulticlassClassificationEvaluator(labelCol="legitimate", predictionCol="prediction")
evaluator = BinaryClassificationEvaluator(labelCol="legitimate", rawPredictionCol="prediction", metricName='areaUnderROC')

predictionAndTarget = dtModel.transform(test_df).select("legitimate", "prediction")

acc = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "accuracy"})
f1 = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "f1"})
weightedPrecision = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedPrecision"})
weightedRecall = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedRecall"})
auc = evaluator.evaluate(predictionAndTarget)

print("accuracy score %.2f" % acc)
print("f1 score %.2f" % f1)
print("weighted precision %.2f" % weightedPrecision)
print("weighted recall score %.2f" % weightedRecall)
print("area under roc score %.2f" % auc)

accuracy score 0.97
f1 score 0.97
weighted precision 0.97
weighted recall score 0.97
area under roc score 0.96


In [24]:
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes(smoothing=1.0, modelType="multinomial", labelCol="legitimate", featuresCol="features")

nbModel = nb.fit(train_df)

In [25]:
evaluatorMulti = MulticlassClassificationEvaluator(labelCol="legitimate", predictionCol="prediction")
evaluator = BinaryClassificationEvaluator(labelCol="legitimate", rawPredictionCol="prediction", metricName='areaUnderROC')

predictionAndTarget = nbModel.transform(test_df).select("legitimate", "prediction")

acc = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "accuracy"})
f1 = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "f1"})
weightedPrecision = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedPrecision"})
weightedRecall = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedRecall"})
auc = evaluator.evaluate(predictionAndTarget)

print("accuracy score %.2f" % acc)
print("f1 score %.2f" % f1)
print("weighted precision %.2f" % weightedPrecision)
print("weighted recall score %.2f" % weightedRecall)
print("area under roc score %.2f" % auc)

accuracy score 0.70
f1 score 0.61
weighted precision 0.79
weighted recall score 0.70
area under roc score 0.57
